In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains

import time

In [8]:
def labe_find(str_find):
    for p in range(len(paging.find_all('span',class_='list__label'))):
        if str_find in paging.find_all('span',class_='list__label')[p].text.strip():
            return paging.find_all('span',class_='list__content')[p].text.strip()
        
def open_page(block):   
        xp = browser.find_element_by_xpath(f'/html/body/div[7]/div[2]/div[1]/div[4]/section[{block}]/a')
        ActionChains(browser).click(xp).perform() #開啟分頁
        time.sleep(3)
        browser.switch_to.window(browser.window_handles[1]) 
        p = browser.page_source #取得網址程式碼
        paging = BeautifulSoup(p)
        return(paging)
    
def open_driver(item):
    browser.get(f'https://www.rakuya.com.tw/sell/result?city=8&sort=11&browsed=0&page={item}')
    r = browser.page_source
    web2 = BeautifulSoup(r)
    return(web2)

In [9]:
#讓瀏覽器在背景執行
options = Options()
#     options.add_argument('--headless')
#用來規避bug
options.add_argument('--disable-gpu')
#以最高權限執行
options.add_argument('--no-sandbox')
#預防資源崩潰
options.add_argument('--disable-dev-shm-usage')

#模擬開啟一個瀏覽器
browser = webdriver.Chrome(options=options, executable_path='./chromedriver')

item = 1
web2 = open_driver(item)
rakuya = []
total_page = int(web2.find('p',class_='pages').text.split('/')[1][:-1].strip())
# browser.implicitly_wait(6) #等待六秒

while item <= total_page:    
    block = 1
    for web in web2.find_all('a', class_='browseItemDetail'):
        if block>19:
            break
        else:
            try:
                paging = open_page(block)
            except:
                browser.quit()
                browser = webdriver.Chrome(options=options, executable_path='./chromedriver')
                web2 = open_driver(item)
                paging = open_page(block)

            try:
                paging.find('span',class_='title').text.strip()
            except:
                browser.get(web.get('href'))
                p = browser.page_source
                paging = BeautifulSoup(p)
                
            rakuya.append({
                    '標題': paging.find('span',class_='title').text.strip(),
                    '建案座落位置': paging.find('h1',class_='txt__address').text.strip(),
                    '建物': labe_find('建物登記'),
                    '主建物': labe_find('主建物'),
                    '附屬建物': labe_find('附屬建物'),
                    '公共設施' : labe_find('公共設施'),
                    '土地登記' : labe_find('土地登記'),
                    '配置':paging.find_all('ul',class_='list__info')[0].find_all('li')[0].get_text()[0:6],
                    '年份': paging.find_all('ul',class_='list__info')[0].find_all('li')[2].get_text()[:-2],
                    '樓層':web.find_all('ul',class_='list__info')[0].find_all('li')[-1].get_text(),
                    '車位與否':paging.find_all('div',class_='list__info-sub')[2].find('span',class_='list__content').text.strip(),
                    '開價': paging.find('span',class_='txt__price-total').text[:-1].strip(),
                    '特色描述' : paging.find('div',class_='block__info-detail').find('div',class_='content').text.strip().replace(u'\xa0',u'') if paging.find('div',class_='block__info-detail').find('div',class_='content') else None
            })
            block+=1
            browser.close() #關閉分頁
            browser.switch_to.window(browser.window_handles[0])
            
    browser.quit()
    item+=1
    browser = webdriver.Chrome(options=options, executable_path='./chromedriver')
    open_driver(item)
    time.sleep(1)
#關閉瀏覽器
browser.quit()

KeyboardInterrupt: 

In [11]:
df = pd.DataFrame(rakuya)
df['建案座落位置']=df['建案座落位置'].str.split('/').str.get(0)
df['行政區'] = df['建案座落位置'].str[3:].str.split('區').str.get(0)+'區'

In [12]:
df

,標題,建案座落位置,建物,主建物,附屬建物,公共設施,土地登記,配置,年份,樓層,車位與否,開價,特色描述,行政區
0,X-專任-大里夜景視野戶三房附車位~成家首選,台中市大里區中湖路,33.48坪,20.47坪,3.34坪,7.55坪,4.89坪,3房2廳2衛,16年,11樓/13樓,無車位,788,"**24H全天候管理,住戶單純,環境佳**高樓景觀視野好，採光通風佳**震後三房兩衛稀有釋出...",大里區
1,教育大學精裝視野美套,台中市北區中山路,9.49坪,5.45坪,1.33坪,None,None,1房1衛格局,27年,7樓/9樓,無車位,388,1.\n全新裝潢，一卡皮箱直接入住2.\n格局方正，視野好無遮蔽3.\n系統櫥櫃，收納多，全...,北區
2,3+4樓公寓,台中市大肚區中沙路,35坪,30坪,None,None,None,3房1廳1.,42年,3樓/3樓,無車位,6860000,None,大肚區
3,南屯五期近捷運電梯漂亮三房+車位,台中市南屯區東興路二段,33.98坪,21.59坪,2.37坪,None,None,3房2廳2衛,28年,4樓/5樓,有車位,888,屋主誠心要出售南屯正五期精華地點 永春國小明星學區有電梯的華廈 樓層是很漂亮的次頂樓還有附車...,南屯區
4,獨家!河南路文華高中旁捷運前院停車美墅,台中市西屯區河南路一段,76.6坪,71.4坪,5.2坪,None,24.3坪,6房3廳5衛,19年,10樓/12樓,有車位,2880,Google台中房仲顧問 廖敏雄 0958588898廖敏雄☎️服務專線:0958-588-...,西屯區
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,中信科博館 錦德名廈四房雙車,台中市北區興進路,107.18坪,58.85坪,7.55坪,31.72坪,10.08坪,4房2廳2衛,15年,4樓/10樓,有車位,2480,物 件 特 色\t\t\t\t※面興進綠園道，高樓層無限視野。\t\t\t\t※下樓即綠園道...,北區
1106,@G6捷運旁北平商圈大坪數三房美寓,台中市北屯區北平路三段,31.92坪,30.55坪,1.37坪,None,8.24坪,3房2廳2衛,38.5年,10樓/12樓,無車位,688,1. 北平、天津雙商圈，生活機能強。\n\n\n2. G6捷運、北平黃昏市場步行4分鐘。\n...,北屯區
1107,@超便宜3房無限視野,台中市北屯區崇德六路一段,35.67坪,23.49坪,3.93坪,3.45坪,5.25坪,3房2廳2衛,31.9年,14樓/14樓,有車位,888,低總價給你大大享受~~~\n\n鬧中取靜\n\n就是這間了!!!!!!!!!\n\n高樓層無...,北屯區
1108,@*近一中5.1%高投報*金雞母全新裝潢獨洗獨曬,台中市北區益華街,55.22坪,45.16坪,6.09坪,3.97坪,9.38坪,7房7衛格局,32.5年,2樓/8樓,無車位,2198,1.近一中、崇德商圈，市場、全聯及便利商店生活便利採買\n\n2.有電梯合法格局，一層一戶整...,北區
